In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import pandas_ta as ta
from urllib.request import urlretrieve
from datetime import datetime

In [2]:
url= 'https://archives.nseindia.com/content/indices/ind_nifty500list.csv'

In [3]:
# there are two reviews: March and September
update= False

# next csv file name: '2024_march.csv'
if update == True:
    urlretrieve(url, 'stocks_review/2023_september.csv')
else:
    pass

In [4]:
# reading the csv file
def listed_stocks(csv):
    split_csv= csv.split('.')[0]

    try:
        return pd.read_csv(f'stocks_review/{split_csv}.csv').iloc[:,:3]
        
    except FileNotFoundError:
        return None

In [5]:
# current stock list csv in use
listed_stocks= listed_stocks('2023_september')

In [6]:
# this cell is used while testing
test_flag= True

if test_flag == True and listed_stocks is not None:
    listed_stocks= listed_stocks.iloc[:5]
else:
    pass

In [7]:
def fetch_data(D):
    # symbols list
    symbols = [symbol+'.NS' for symbol in listed_stocks.Symbol]

    dataframes = []
    # fetching data
    for symbol in symbols:
        try:
            data = yf.Ticker(symbol).history(period=str(D)+'d')
            data['Symbol'] = symbol.split('.')[0]
            dataframes.append(data)
        except IndexError:
            print(f'error fetching data for {symbol}: {IndexError}')

    if not dataframes:
        return None
    
    # putting collected data in dataframe
    historical_data= pd.concat(dataframes)
    historical_data.reset_index(inplace=True)

    # optimising dataframe
    historical_data = historical_data.round(2)
    historical_data.drop(columns=['Dividends', 'Stock Splits'], inplace=True)
    historical_data['Date'] = pd.to_datetime(historical_data['Date']).dt.strftime('%Y-%m-%d')
    historical_data.set_index('Date', inplace=True)
    
    return historical_data

In [8]:
def fetch_price_band(D):
    data= fetch_data(D)
    grouped_data= data.groupby('Symbol')
    
    # values for calculations
    min_price= grouped_data.Low.min()
    max_price= grouped_data.High.max()
    avg_price= grouped_data.Close.mean()
    
    # calculation
    price_range= (max_price-min_price)/avg_price
    
    # assigning the calculated result to a dataframe
    price_range_data= pd.Series(price_range, name=f'Price Deviation ({D} D)').round(5)
    price_range_data= price_range_data.to_frame()
    price_range_data.reset_index(inplace= True)
    
    # creating merged dataframe
    merged_dataframe= pd.merge(listed_stocks, price_range_data, on= 'Symbol', how= 'inner')
    merged_dataframe.sort_values(by=f'Price Deviation ({D} D)', inplace=True)
    merged_dataframe.reset_index(inplace= True, drop= True)
    
    return merged_dataframe

In [9]:
def symbol_price_action(symbol):
    dataframe= pd.DataFrame()
    
    data= yf.Ticker(symbol+'.NS').history(period='1750d').Close.to_frame()
    data.index= data.index.date.astype('datetime64')
    data= data.round(2)

    dataframe['LTP']= data.iloc[-1]  # Latest data-point
    
    # short term
    if len(data) >= 125:
        dataframe['ST']= data.iloc[-125]   # 125 days ago
    else:
        dataframe['ST']= dataframe.LTP

    # medium term
    if len(data) >= 500:
        dataframe['MT']= data.iloc[-500]   # 500 days ago
    else:
        dataframe['MT']= dataframe.ST

    # long term
    if len(data) >= 1250:
        dataframe['LT']= data.iloc[-1250]  # 1250 days ago
    else:
        dataframe['LT']= dataframe.MT
    
    dataframe['ST %']= ((dataframe.LTP - dataframe.ST) * 100 / dataframe.LTP).round(3)
    dataframe['MT %']= ((dataframe.LTP - dataframe.MT) * 100 / dataframe.LTP).round(3)
    dataframe['LT %']= ((dataframe.LTP - dataframe.LT) * 100 / dataframe.LTP).round(3)

    dataframe['Symbol']= symbol
    dataframe.reset_index(inplace=True, drop=True)
    
    return dataframe

In [10]:
def fetch_price_action():
    dataframe= pd.DataFrame()
    for symbol in listed_stocks['Symbol']:
        symbol_data= symbol_price_action(symbol)
        dataframe= pd.concat([dataframe, symbol_data], ignore_index= True)
    return dataframe

In [11]:
def fetch_data_details(D):
    price_band= fetch_price_band(D)
    price_action= fetch_price_action()

    merged_dataframe= pd.merge(price_band, price_action, on= 'Symbol', how='inner')

    return merged_dataframe

In [12]:
def fetch_stop_loss_and_target(symbol, risk_reward_ratio=2, atr_period=14, atr_multiplier=3, rsi_period=14):
    symbol_ns = symbol + '.NS'
    data = yf.Ticker(symbol_ns).history(period=str(max(atr_period, rsi_period)) + 'd').round(2).iloc[:, :4]
    data.index = data.index.date.astype('datetime64')

    # calculating average true range
    C = data['Close'].to_numpy()
    H = data['High'].to_numpy()
    L = data['Low'].to_numpy()

    H_L= H - L
    H_PDC= np.abs(H - np.roll(C, 1))
    L_PDC= np.abs(L - np.roll(C, 1))

    TR= np.maximum.reduce([H_L, H_PDC, L_PDC])

    ATR= np.zeros(len(TR))
    ATR[atr_period - 1]= np.mean(TR[:atr_period])
    for i in range(atr_period, len(TR)):
        ATR[i]= (ATR[i-1]*(atr_period-1) + TR[i])/atr_period

    data[f'ATR ({atr_period})']= ATR
        
    # calculating relative strength index
    delta= np.diff(data.Close, prepend= np.nan)
    
    gain= np.where(delta > 0,  delta, 0)
    loss= np.where(delta < 0, -delta, 0)
    
    avg_gain= np.convolve(gain, np.ones(rsi_period)/rsi_period, mode= 'valid')
    avg_loss= np.convolve(loss, np.ones(rsi_period)/rsi_period, mode= 'valid')

    rs= avg_gain/avg_loss

    rsi= 100-(100/(1+rs))
    rsi= np.concatenate((np.full(rsi_period - 1, np.nan), rsi))

    data[f'RSI ({rsi_period})']= rsi
    
    # calculating stop loss and target
    data['SL'] = data.Close - data[f'ATR ({atr_period})'] * atr_multiplier
    data['Target'] = data.Close + data[f'ATR ({atr_period})'] * atr_multiplier * risk_reward_ratio

    resultant = data.tail(1).copy()
    resultant = resultant.rename_axis(f'R : R Ratio = {risk_reward_ratio}')
    resultant = resultant.round(2)
    resultant.index = resultant.index.strftime('%Y-%m-%d')

    return resultant

In [13]:
fetch_stop_loss_and_target('PIDILITIND')

,Open,High,Low,Close,ATR (14),RSI (14),SL,Target
R : R Ratio = 2,,,,,,,,
2023-09-27,2495.0,2503.75,2488.0,2488.0,32.68,44.1,2389.95,2684.09


In [14]:
#obtain_data_csv(5)